In [2]:
import numpy as np
import pandas as pd
import requests
import datetime
import calendar
import scipy.stats as stats
import matplotlib.pyplot as plt


In [3]:
import os
from eod import EodHistoricalData

In [4]:
# Load the key from the environment variables
api_key = os.environ.get('63ef81c909b516.05540383')

# Create the client instance
client = EodHistoricalData(api_key)

In [5]:
from eod import EodHistoricalData
# create the instance of the SDK
api_key = '63ef81c909b516.05540383'
client = EodHistoricalData(api_key)

In [6]:
resp = client.get_prices_intraday('EUR.FOREX', interval='1m', from_='1620136800', to='1620414000')

In [7]:
import pandas as pd

data = [{'timestamp': r['timestamp'],
         'datetime': r['datetime'],
         'open': r['open'],
         'high': r['high'],
         'low': r['low'],
         'close': r['close'],
         'volume': r['volume']} for r in resp]

df = pd.DataFrame(data)

# set the datetime column as the index
df.set_index('datetime', inplace=True)

print(df.head())

                      timestamp     open     high      low    close  volume
datetime                                                                   
2021-05-04 14:00:00  1620136800  1.20239  1.20250  1.20186  1.20186     220
2021-05-04 14:01:00  1620136860  1.20190  1.20215  1.20180  1.20205     170
2021-05-04 14:02:00  1620136920  1.20205  1.20216  1.20161  1.20172     205
2021-05-04 14:03:00  1620136980  1.20175  1.20204  1.20160  1.20204     156
2021-05-04 14:04:00  1620137040  1.20203  1.20236  1.20192  1.20227     163


In [8]:
resp = client.get_prices_intraday('MSFT.US', interval='1m', from_='1577836800', to='1583020800')

In [9]:
df

,timestamp,open,high,low,close,volume
datetime,,,,,,
2021-05-04 14:00:00,1620136800,1.20239,1.20250,1.20186,1.20186,220
2021-05-04 14:01:00,1620136860,1.20190,1.20215,1.20180,1.20205,170
2021-05-04 14:02:00,1620136920,1.20205,1.20216,1.20161,1.20172,205
2021-05-04 14:03:00,1620136980,1.20175,1.20204,1.20160,1.20204,156
2021-05-04 14:04:00,1620137040,1.20203,1.20236,1.20192,1.20227,163
...,...,...,...,...,...,...
2021-05-07 18:56:00,1620413760,1.21636,1.21642,1.21620,1.21640,82
2021-05-07 18:57:00,1620413820,1.21636,1.21645,1.21620,1.21645,67
2021-05-07 18:58:00,1620413880,1.21640,1.21650,1.21620,1.21643,82


In [10]:
#All intraday data as csv files 
#Check intraday data for completeness 
#Summarize ftx data per minute -> round if necessary -> 00:02:20 is 00:03:00
#Price Quality 

In [19]:
import pandas as pd
from datetime import datetime, timedelta

# specify the start and end date for the data
start_date = datetime(2020, 1, 1, 0, 1)
end_date = datetime(2022, 12, 31, 23, 59)

# specify the interval for the data
interval = '1m'

# initialize an empty list to store the data
data = []

# define a function to fetch data in chunks
def fetch_data(from_date, to_date):
    while from_date < end_date:
        # get the 'to' date by adding 120 days to the 'from' date
        to_date = from_date + timedelta(days=120)

        # convert the 'from' and 'to' dates to the required format for the API call
        from_str = str(int(from_date.timestamp()))
        to_str = str(int(to_date.timestamp()))

        # make the API call
        try:
            resp = client.get_prices_intraday('MSFT.US', interval=interval, from_=from_str, to=to_str)
        except V20Error as e:
            print(f"Error: {e}")
            break

        # convert the response data to a list of dictionaries
        data.extend([{'datetime': r['datetime'], 'open': r['open'], 'high': r['high'], 'low': r['low'], 'close': r['close'], 'volume': r['volume']} for r in resp])

        # set the 'from' date for the next iteration to be 1 second after the 'to' date of the current iteration
        from_date = to_date + timedelta(seconds=1)

# fetch the data in chunks and store in the list 'data'
fetch_data(start_date, start_date)

# convert the list of dictionaries to a dataframe
df = pd.DataFrame(data)

# set the datetime column as the index
df.set_index('datetime', inplace=True)

# save the dataframe to a CSV file
df.to_csv('msft_data.csv')


In [20]:
df

,open,high,low,close,volume
datetime,,,,,
2019-12-31 23:02:00,157.83,157.83,157.83,157.83,300
2019-12-31 23:16:00,157.82,157.82,157.82,157.82,150
2019-12-31 23:23:00,157.81,157.81,157.81,157.81,362
2019-12-31 23:41:00,157.81,157.81,157.81,157.81,1080
2019-12-31 23:49:00,157.81,157.81,157.81,157.81,265
...,...,...,...,...,...
2023-02-25 00:36:00,248.90,248.90,248.90,248.90,299
2023-02-25 00:39:00,249.17,249.17,249.17,249.17,384
2023-02-25 00:51:00,248.86,248.86,248.86,248.86,143


In [23]:
import pandas as pd
from datetime import datetime, timedelta

# define a function to fetch data in chunks
def fetch_data(client, symbol, market, start_date, end_date, interval):
    # initialize an empty list to store the data
    data = []
    from_date = start_date
    to_date = from_date + timedelta(days=120)
    while from_date < end_date:
        # convert the 'from' and 'to' dates to the required format for the API call
        from_str = str(int(from_date.timestamp()))
        to_str = str(int(to_date.timestamp()))

        # make the API call
        try:
            resp = client.get_prices_intraday(f"{symbol}.{market}", interval=interval, from_=from_str, to=to_str)
        except V20Error as e:
            print(f"Error: {e}")
            break

        # convert the response data to a list of dictionaries
        data.extend([{'datetime': r['datetime'], 'open': r['open'], 'high': r['high'], 'low': r['low'], 'close': r['close'], 'volume': r['volume']} for r in resp])

        # set the 'from' date for the next iteration to be 1 second after the 'to' date of the current iteration
        from_date = to_date + timedelta(seconds=1)
        to_date = from_date + timedelta(days=120)

    # convert the list of dictionaries to a dataframe
    df = pd.DataFrame(data)

    # set the datetime column as the index
    df.set_index('datetime', inplace=True)

    # save the dataframe to a CSV file with a filename that includes the symbol and market
    filename = f"{symbol}_{market}_data.csv"
    df.to_csv(filename)
    print(f"Data saved to {filename}")

# specify the start and end date for the data
start_date = datetime(2020, 1, 1, 0, 1)
end_date = datetime(2022, 12, 31, 23, 59)

# specify the interval for the data
interval = '1m'

# fetch the data and save to a CSV file
fetch_data(client, 'MSFT', 'US', start_date, end_date, interval)


Data saved to MSFT_US_data.csv


In [24]:
fetch_data(client, 'AAPL', 'US', start_date, end_date, interval)

Data saved to AAPL_US_data.csv


In [25]:
fetch_data(client, 'ABNB', 'US', start_date, end_date, interval)

Data saved to ABNB_US_data.csv


In [26]:
fetch_data(client, 'ACB', 'US', start_date, end_date, interval)

Data saved to ACB_US_data.csv


In [27]:
fetch_data(client, 'AMC', 'US', start_date, end_date, interval)

Data saved to AMC_US_data.csv


In [28]:
fetch_data(client, 'AMZN', 'US', start_date, end_date, interval)

Data saved to AMZN_US_data.csv


In [29]:
fetch_data(client, 'APHA', 'US', start_date, end_date, interval)

Data saved to APHA_US_data.csv


In [30]:
fetch_data(client, 'ARKK', 'US', start_date, end_date, interval)

Data saved to ARKK_US_data.csv


In [31]:
fetch_data(client, 'BABA', 'US', start_date, end_date, interval)

Data saved to BABA_US_data.csv


In [32]:
fetch_data(client, 'BB', 'US', start_date, end_date, interval)

Data saved to BB_US_data.csv


In [33]:
fetch_data(client, 'BILI', 'US', start_date, end_date, interval)

Data saved to BILI_US_data.csv


In [34]:
fetch_data(client, 'BITO', 'US', start_date, end_date, interval)

Data saved to BITO_US_data.csv


In [36]:
fetch_data(client, 'BITW', 'US', start_date, end_date, interval)

KeyError: "None of ['datetime'] are in the columns"

In [37]:
fetch_data(client, 'BNTX', 'US', start_date, end_date, interval)

Data saved to BNTX_US_data.csv


In [38]:
fetch_data(client, 'BYND', 'US', start_date, end_date, interval)

Data saved to BYND_US_data.csv


In [40]:
fetch_data(client, 'CGC', 'US', start_date, end_date, interval)

Data saved to CGC_US_data.csv


In [41]:
fetch_data(client, 'COIN', 'US', start_date, end_date, interval)

Data saved to COIN_US_data.csv


In [42]:
fetch_data(client, 'CRON', 'US', start_date, end_date, interval)

Data saved to CRON_US_data.csv


In [43]:
fetch_data(client, 'DKNG', 'US', start_date, end_date, interval)

Data saved to DKNG_US_data.csv


In [44]:
fetch_data(client, 'ETHE', 'US', start_date, end_date, interval)

KeyError: "None of ['datetime'] are in the columns"

In [1]:
fetch_data(client, 'META', 'US', start_date, end_date, interval)

NameError: name 'fetch_data' is not defined

In [46]:
fetch_data(client, 'GBTC', 'US', start_date, end_date, interval)

KeyError: "None of ['datetime'] are in the columns"

In [47]:
fetch_data(client, 'GDXJ', 'US', start_date, end_date, interval)

Data saved to GDXJ_US_data.csv


In [48]:
fetch_data(client, 'GDX', 'US', start_date, end_date, interval)

Data saved to GDX_US_data.csv


In [49]:
fetch_data(client, 'GLD', 'US', start_date, end_date, interval)

Data saved to GLD_US_data.csv


In [50]:
fetch_data(client, 'GLXY', 'US', start_date, end_date, interval)

KeyError: "None of ['datetime'] are in the columns"

In [51]:
fetch_data(client, 'GME', 'US', start_date, end_date, interval)

Data saved to GME_US_data.csv


In [52]:
fetch_data(client, 'GOOGL', 'US', start_date, end_date, interval)

Data saved to GOOGL_US_data.csv


In [53]:
fetch_data(client, 'HOOD', 'US', start_date, end_date, interval)

Data saved to HOOD_US_data.csv


In [54]:
fetch_data(client, 'MRNA', 'US', start_date, end_date, interval)

Data saved to MRNA_US_data.csv


In [55]:
fetch_data(client, 'MSTR', 'US', start_date, end_date, interval)

Data saved to MSTR_US_data.csv


In [39]:
fetch_data(client, 'NFLX', 'US', start_date, end_date, interval)

Data saved to NFLX_US_data.csv


In [56]:
fetch_data(client, 'NIO', 'US', start_date, end_date, interval)

Data saved to NIO_US_data.csv


In [57]:
fetch_data(client, 'NOK', 'US', start_date, end_date, interval)

Data saved to NOK_US_data.csv


In [58]:
fetch_data(client, 'NVDA', 'US', start_date, end_date, interval)

Data saved to NVDA_US_data.csv


In [59]:
fetch_data(client, 'PENN', 'US', start_date, end_date, interval)

Data saved to PENN_US_data.csv


In [60]:
fetch_data(client, 'PFE', 'US', start_date, end_date, interval)

Data saved to PFE_US_data.csv


In [61]:
fetch_data(client, 'PYPL', 'US', start_date, end_date, interval)

Data saved to PYPL_US_data.csv


In [62]:
fetch_data(client, 'SLV', 'US', start_date, end_date, interval)

Data saved to SLV_US_data.csv


In [63]:
fetch_data(client, 'SPY', 'US', start_date, end_date, interval)

Data saved to SPY_US_data.csv


In [64]:
fetch_data(client, 'SQ', 'US', start_date, end_date, interval)

Data saved to SQ_US_data.csv


In [65]:
fetch_data(client, 'TLRY', 'US', start_date, end_date, interval)

Data saved to TLRY_US_data.csv


In [66]:
fetch_data(client, 'TSLA', 'US', start_date, end_date, interval)

Data saved to TSLA_US_data.csv


In [67]:
fetch_data(client, 'TWTR', 'US', start_date, end_date, interval)

Data saved to TWTR_US_data.csv


In [68]:
fetch_data(client, 'UBER', 'US', start_date, end_date, interval)

Data saved to UBER_US_data.csv


In [69]:
fetch_data(client, 'USO', 'US', start_date, end_date, interval)

Data saved to USO_US_data.csv


In [70]:
fetch_data(client, 'WNDR', 'US', start_date, end_date, interval)

NameError: name 'V20Error' is not defined

In [71]:
fetch_data(client, 'ZM', 'US', start_date, end_date, interval)

Data saved to ZM_US_data.csv
